In [2]:
# Import libararies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


# Load the dataset
df = pd.read_csv('2019-11-13-11-57_2331.csv', encoding='ISO-8859-1')
df.head()

,Timestamp,STATION_1:Freq,STATION_1:Va:Magnitude,STATION_1:Va:Angle,STATION_1:Vb:Magnitude,STATION_1:Vb:Angle,STATION_1:Vc:Magnitude,STATION_1:Vc:Angle,STATION_1:SlewRate,STATION_1:OFDetect,STATION_1:UFDetect,STATION_1:Slew50,STATION_1:Slew100,STATION_1:Slew200,STATION_1:OFSlew,STATION_1:UFSlew,STATION_1:DF/DT
0,2019/11/13 11:52:49.467,60.00009,121.31514,121.84017,121.92584,1.734281,121.43974,-118.45382,0.000078,0,0,0.001449173,0.001153005,0.0007983182,0,0,0.008190001
1,2019/11/13 11:52:49.483,60.00009,121.31514,121.84017,121.92584,1.734281,121.43974,-118.45382,0.000110,0,0,0.001553219,0.001184502,0.0008279819,0,0,0.008190001
2,2019/11/13 11:52:49.499,59.99945,121.30986,121.83616,121.91555,1.729726,121.43974,-118.45382,0.000126,0,0,0.001657264,0.00123889,0.0008233126,0,0,-0.00116
3,2019/11/13 11:52:49.515,59.99945,121.30986,121.83616,121.91555,1.729726,121.43974,-118.45382,0.000108,0,0,0.00155212,0.001311407,0.0008186433,0,0,-0.00116
4,2019/11/13 11:52:49.531,59.99818,121.31104,121.81914,121.91435,1.710936,121.42767,-118.47300,0.000100,0,0,0.001538565,0.001342904,0.000813974,0,0,-0.00639


In [18]:
# Checking the missing value
df.isna().sum()


# Selecting the target
y = df.iloc[:, 1]
y.head()


# Selecting the features
X = df.iloc[:,2:9]
X['STATION_1:DF/DT'] = df.iloc[:,-1]
X.head()


# There is a wrong data type
# Checking the dtype
X.info()

# DF/DT columns is set as a object
# Convert it into float
obj_new_value = []
for obj in X.iloc[:,-1]:
    if obj[-1] == 'K':
        obj.strip('K')
        obj_new_value.append(obj)
    else:
        obj_new_value.append(obj)
obj_new_value

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18000 entries, 0 to 17999
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   STATION_1:Va:Magnitude  18000 non-null  float64
 1   STATION_1:Va:Angle      18000 non-null  float64
 2   STATION_1:Vb:Magnitude  18000 non-null  float64
 3   STATION_1:Vb:Angle      18000 non-null  float64
 4   STATION_1:Vc:Magnitude  18000 non-null  float64
 5   STATION_1:Vc:Angle      18000 non-null  float64
 6   STATION_1:SlewRate      18000 non-null  float64
 7   STATION_1:DF/DT         18000 non-null  object 
dtypes: float64(7), object(1)
memory usage: 1.1+ MB


['0.008190001',
 '0.008190001',
 '-0.00116',
 '-0.00116',
 '-0.00639',
 '-0.00639',
 '0.00017',
 '0.00017',
 '0.00175',
 '0.00175',
 '0.00175',
 '-0.0074',
 '-0.0074',
 '-0.00034',
 '-0.00034',
 '0.00485',
 '0.00485',
 '2E-05',
 '2E-05',
 '0.00475',
 '0.00475',
 '0.00613',
 '0.00613',
 '-0.00019',
 '-0.00019',
 '0.00445',
 '0.00445',
 '0.00314',
 '0.00314',
 '0.00217',
 '0.00217',
 '-0.00123',
 '-0.00123',
 '0.00228',
 '0.00228',
 '0.00228',
 '0.00145',
 '0.00145',
 '0.00735',
 '0.00735',
 '-0.00322',
 '-0.00322',
 '0.00307',
 '0.00307',
 '0.008070001',
 '0.008070001',
 '-0.00051',
 '-0.00051',
 '-0.00822',
 '-0.00822',
 '0.00544',
 '0.00544',
 '0.00049',
 '0.00049',
 '-0.00383',
 '-0.00383',
 '0.00201',
 '0.00201',
 '0.00053',
 '0.00053',
 '0.00053',
 '0.0',
 '0.0',
 '0.0021',
 '0.0021',
 '-0.0028',
 '-0.0028',
 '-0.00036',
 '-0.00036',
 '0.00656',
 '0.00656',
 '-0.00703',
 '-0.00703',
 '-0.00202',
 '-0.00202',
 '0.00453',
 '0.00453',
 '-0.00306',
 '-0.00306',
 '-0.00302',
 '-0.00302'